In [1]:
from guitarpro.models import GPException
import os, glob, json
import guitarpro

In [2]:
def get_guitar_tracks(song):
    """
    24 Acoustic Guitar (nylon)
    25 Acoustic Guitar (steel)
    26 Electric Guitar (jazz)
    27 Electric Guitar (clean)
    28 Electric Guitar (muted)
    29 Overdriven Guitar
    30 Distortion Guitar
    """
    GUITAR_MIDI_PROGRAMS = [24, 25, 26, 27, 28, 29, 30]
    # get all non-percussive tracks (this is still necessary because some drum tracks use a guitar program number)
    m_tracks = [track for track in song.tracks if not track.isPercussionTrack]
    guitar_tracks = [
        track
        for track in m_tracks
        if track.channel.instrument in GUITAR_MIDI_PROGRAMS and len(track.strings) == 6
    ]
    return guitar_tracks

def get_note_info(note, bpm, margin=None):
    """
    This is the comprehensive function for generating note-level annotation

    It calls `get_note_time` and `get_effect_info`

    `margin` is for passing in the global onset of the segment and calculate the note start time in the segment
    """
    def get_effect_info(effect):
        effect_info = {
            "bend": bool(effect.isBend),  # bool,
            "vibrato": effect.vibrato,  # bool
            "hammer": effect.hammer,  # bool
            "slide": bool(effect.slides),  # bool
        }
        effect_info["bend_type"] = effect.bend.type.name if effect.isBend else None
        effect_info["slide_types"] = (
            [slide.name for slide in effect.slides] if effect.slides else None
        )
        return effect_info

    def get_note_time(note, bpm, margin=None):
        start = note.beat.start
        start_sec = round(((start - 960) / 960) / (bpm / 60), 4)
        # the note timing info encoded in a GP file is global, i.e., the start time in the song
        # I want the start time in the segment, `margin` is the start time of the segment
        if margin:
            start_sec = start_sec - margin
        dur = note.beat.duration.time
        dur_sec = round((dur / 960) / (bpm / 60), 4)
        time = {"start": start_sec, "dur": dur_sec}
        return time

    note_info = {
        "time": get_note_time(note, bpm, margin=margin),
        "string": note.string,
        "fret": note.value,  # fret number
        # "dur_percent": note.durationPercent,
        "pitch": note.realValue,  # self.value + string.value = MIDI note number
        # "type": note.type.name,  # NoteType class, rest=0, normal=1, tie=2, dead=3
        "effects": get_effect_info(note.effect),
    }
    return note_info

In [39]:
def get_single_tracks(
    file,
    output_dir,
    unify_volume=True,
    force_clean=True,
    disable_repeats=True,
    disable_mixTableChange=True,
    disable_other_techniques=True,
    force_normal=True
):
    """Split one multi-track GuitarPro file into several one-track GuitarPro files

    Args:
        file (str): The path to the file to split
        output_dir (str): The directory for the output files
        unify_volume (bool, optional): Whether to adjust the volume of every track to the same level. Defaults to True.
        force_clean (bool, optional): Whether to force all tracks to use the clean electric guitar tone. Defaults to True.
        disable_repeats (bool, optional): Whether to disable all repeats and alternate endings in the GuitarPro file. Defaults to True.
        disable_mixTableChange (bool, optional): Whether to disable mixTableChange instances (e.g., tempo change in the middle of the song). Defaults to True.
        disable_other_techniques (bool, optional): Whether to disable other playing techniques (e.g., grace notes, slidein, slideout). Defaults to True.
        force_normal (bool, optional): Whether to change all note types (rest, dead, tie) to normal. Defaults to True.
    """
    song = guitarpro.parse(file)
    tracks = get_guitar_tracks(song)
    for track in tracks:
        # unify the volume for rendered audio
        if unify_volume:
            # default volume is 120 for newly created tracks
            track.channel.volume = 120
        # force the instrument to be clean electric guitar, so that synthesized audio is automatically clean guitar
        if force_clean:
            track.channel.instrument = 27

        # disable repeats in all measures
        # this includes repeats and alternative endings
        for measure in track.measures:
            if disable_repeats:
                # isRepeatOpen is boolean, repeatClose takes -1 or 1,
                # repeatAlternative can be whatever number, depending on which repeat group it belongs to
                # the following is the default setting in normal bars
                measure.header.isRepeatOpen = False
                measure.header.repeatClose = -1
                measure.header.repeatAlternative = 0
            # disable mixTableChange in all beats
            # this includes tempo changes, which mess up the calculation of note timings
            # and other mysterious effect/instrument changes
            if disable_mixTableChange:
                for voice in measure.voices:
                    for beat in voice.beats:
                        beat.effect.mixTableChange = None
            
            if disable_other_techniques:
                for voice in measure.voices:
                    for beat in voice.beats:
                        beat.effect.fadeIn = False
                        beat.effect.tremoloBar = None
                        for note in beat.notes:
                            note.effect.grace = None
                            note.effect.harmonic = None
                            note.effect.trill = None
                            # slideType values: 1 - shiftSlideTo, 2 - legatoSlideTo, others are ignored
                            note.effect.slides = [slide for slide in note.effect.slides if slide.value in [1, 2]]
                            note.effect.letRing = False

            # force_normal will change the NoteType (tie, rest, dead) to normal,
            # tied notes are very messy and hard to handle, so I choose to turn this on
            if force_normal:
                for voice in measure.voices:
                    for beat in voice.beats:
                        for note in beat.notes:
                            note.type = NoteType.normal

        single_track_song = song  # this preserves the metadata in orginal song
        single_track_song.tracks = [track]
        file_name = "{}_{}.gp5".format(
            file.split("/")[-1].split(".")[0], track.name.replace("/", " ")
        )
        try:
            guitarpro.write(single_track_song, os.path.join(output_dir, file_name))
        except GPException:
            print(f"GPException, removing the corrupt file {file_name}")
            os.remove(os.path.join(output_dir, file_name))


In [16]:
MULTI_TRACK_GTP_DIR = "/Volumes/MacOnly/UG_raw/all_time_top_by_hits"
SINGLE_TRACK_GTP_DIR = "/Volumes/MacOnly/UG_rewrite/all_time_top_by_hits/clean_single_track_gtps"
SINGLE_TRACK_ANNO_DIR = "/Volumes/MacOnly/UG_rewrite/all_time_top_by_hits/clean_single_track_annos"

In [ ]:
# generate single track gtps from multi track gtps
i = 0
for file in glob.glob(os.path.join(MULTI_TRACK_GTP_DIR, "*.gp*")):
    print(i)
    get_single_tracks(file, output_dir=SINGLE_TRACK_GTP_DIR, unify_volume=True, force_clean=True, disable_repeats=True, disable_mixTableChange=True, disable_other_techniques=True)
    i += 1

In [42]:
# check that there are no notes in the second voice of each measure
for file in glob.glob(os.path.join(SINGLE_TRACK_GTP_DIR, "*.gp5")):
    song = guitarpro.parse(file)
    track = song.tracks[0]
    for measure in track.measures:
        for beat in measure.voices[1].beats:
            if beat.notes:
                print(file)
                break

In [43]:
# check that every bar has at least one beat, even when it's complete silence
for file in glob.glob(os.path.join(SINGLE_TRACK_GTP_DIR, "*.gp5")):
    song = guitarpro.parse(file)
    track = song.tracks[0]
    for measure in track.measures:
        if len(measure.voices[0].beats) == 0:
            print(file)
            break

In [11]:
def poly_vs_mono_vs_silence(song):
    """Return the time stamps for the start and end of each monophonic / polyphonic / silence segments in the song

    Args:
        song (Song): A pyguitarpro Song object. The song to analyze

    Returns:
        list, list, list: A list of (start, end) time stamps for all mono segments, poly segments, and silence segments
    """
    bpm = song.tempo
    poly_segments = []
    mono_segments = []
    silence_segments = []

    previous_beat_status = -1
    beats = []
    for measure in song.tracks[0].measures:
        voice = measure.voices[0]
        beats.extend(voice.beats)
    for beat in beats:
        onset = beat.start
        onset_sec = round(((onset - 960) / 960) / (bpm / 60), 4)
        dur = beat.duration.time
        dur_sec = round((dur / 960) / (bpm / 60), 4)
        offset_sec = onset_sec + dur_sec
        # 2 for polyphonic, 1 for monophonic, 0 for silence
        if len(beat.notes) == 0:
            beat_status = 0
        elif len(beat.notes) == 1:
            beat_status = 1
        else:
            beat_status = 2
        if beat_status != previous_beat_status:
            # if current beat status is different from the previous beat, add the timing to the output list
            # the following lines can obviously be better written, I leave it like this just for clarity
            if beat_status == 2:
                poly_segments.append([onset_sec, offset_sec])
            elif beat_status == 1:
                mono_segments.append([onset_sec, offset_sec])
            else:
                assert beat_status == 0
                silence_segments.append([onset_sec, offset_sec])
        else:
            # if current beat status is the same as the previous one, update the offset of the previous entry
            if beat_status == 2:
                poly_segments[-1][1] = offset_sec
            elif beat_status == 1:
                mono_segments[-1][1] = offset_sec
            else:
                assert beat_status == 0
                silence_segments[-1][1] = offset_sec
        previous_beat_status = beat_status
    return poly_segments, mono_segments, silence_segments


In [3]:
def gen_anno(file, anno_dir):
    """Generate note-info annotation JSON files, one annotation file per single track GP file

    The input file is a clean single-track GuitarPro file (the whole track). 
    Only mono notes are recorded. 
    
    Args:
        file (str): The path to the single-track GuitarPro file
        anno_dir (str): The directory to put generated JSON file
    """
    song = guitarpro.parse(file)
    bpm = song.tempo
    # put all beats of the song in one place
    beats = []
    for measure in song.tracks[0].measures:
        beats.extend(measure.voices[0].beats)

    note_infos = []

    for beat in beats:
        if len(beat.notes) == 1:
            note = beat.notes[0]
            note_info = get_note_info(note, bpm, margin=None)
            note_infos.append(note_info)

    track_title, _ = os.path.splitext(file.split("/")[-1])
    with open(os.path.join(anno_dir, f"{track_title}.json"), "w") as outfile:
        json.dump(note_infos, outfile, indent=4)


In [ ]:
# generate global annotation from single track gtps
i = 0
for file in glob.glob(os.path.join(SINGLE_TRACK_GTP_DIR, "*.gp5")):
    i += 1
    gen_anno(file, anno_dir=SINGLE_TRACK_ANNO_DIR)
    print(i)

In [12]:
p, m, s = poly_vs_mono_vs_silence(guitarpro.parse(file))

In [15]:
m

[[0.0, 34.375],
 [35.0, 59.375],
 [100.0, 124.375],
 [125.0, 149.375],
 [240.0, 264.375]]